In [1]:
!pwd

/workspace/code/gym-continuousDoubleAuction_agent_based_0/gym_continuousDoubleAuction


In [2]:
%run ../gym_continuousDoubleAuction/test/test_accounting.py

2025-12-22 03:29:16,323	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-12-22 03:29:17,335	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
.............
----------------------------------------------------------------------
Ran 13 tests in 0.008s

OK


In [3]:
%run ../gym_continuousDoubleAuction/test/test_orderBook_old.py

***Bids***
  size price  trade_id  timestamp  order_id
0    5    99       100          5         5
1    5    99       102          7         7
2    5    98       101          6         6
3    5    97       103          8         8
***Asks***
  size price  trade_id  timestamp  order_id
0    5   101       100          1         1
1    5   101       102          3         3
2    5   101       103          4         4
3    5   103       101          2         2
***tape***


{'type': 'limit', 'side': 'bid', 'quantity': 2, 'price': 102, 'trade_id': 109}
Trade occurs as incoming bid limit crosses best ask
[{'timestamp': 9, 'price': Decimal('101'), 'quantity': 2, 'time': 9, 'counter_party': {'ID': 100, 'side': 'ask', 'order_id': 1, 'new_book_quantity': Decimal('3')}, 'init_party': {'ID': 109, 'side': 'bid', 'order_id': None, 'new_book_quantity': None}}]
***Bids***
  size price  trade_id  timestamp  order_id
0    5    99       100          5         5
1    5    99       102          7         7

In [4]:
%run ../gym_continuousDoubleAuction/test/test_orderbook.py

............
----------------------------------------------------------------------
Ran 12 tests in 0.006s

OK



--- Modifying Order ---
Order ID: 1
Order Update: {'side': 'bid', 'quantity': Decimal('10'), 'price': Decimal('101'), 'timestamp': 101}

--- Modifying Order ---
Order ID: 1
Order Update: {'side': 'bid', 'quantity': Decimal('5'), 'price': Decimal('100'), 'timestamp': 101}


In [5]:
%run ../gym_continuousDoubleAuction/test/repro_orderbook_double_delete_order.py

--- Starting Reproduction Script ---
Step 1: Placing initial order at price 100...
Order created with ID: 1

Step 2: Modifying price from 100 to 101...

--- Modifying Order ---
Order ID: 1
Order Update: {'side': 'bid', 'quantity': Decimal('10'), 'price': Decimal('101'), 'timestamp': 2}
Success! (If you see this, the bug is fixed)


In [6]:
%run ../gym_continuousDoubleAuction/test/repro_orderbook_volume_bug.py

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK



--- Testing Partial Fill Volume Sync ---
Placing Ask: 10 units @ 100
Initial OrderTree.volume: 10
Initial Calculated volume: 10

Executing Partial Fill: Bid 4 units @ 100
Final OrderTree.volume: 6 <--- BUG: This should be 6
Final Calculated volume: 6
SUCCESS: Volume is in sync.


In [7]:
%run ../gym_continuousDoubleAuction/test/repro_orderbook_crossed_book.py

--- Starting Crossed Book Reproduction Script ---
Step 1: Placing limit Ask at 100...
Step 2: Placing limit Bid at 90...
Current Best Bid: 90, Best Ask: 100

Step 3: Modifying Bid price from 90 to 110 (Crossing the Ask at 100)...

--- Modifying Order ---
Order ID: 2
Order Update: {'side': 'bid', 'quantity': Decimal('10'), 'price': Decimal('110'), 'timestamp': 2}
Resulting Best Bid: None
Resulting Best Ask: None

Bug not found or already fixed.


In [8]:
%run ../gym_continuousDoubleAuction/test/test_modify_order.py

--- Starting Full 6-Scenario Accounting Verification ---

--- Scenario 1: Price Crosses (10@90 -> 10@110, Ask@100) ---

--- Modifying Order ---
Order ID: 2
Order Update: {'type': 'limit', 'side': 'bid', 'quantity': Decimal('10'), 'price': Decimal('110'), 'trade_id': 'B'}
DEBUG Scenario 1: B Cash=9000, Hold=0, Pos=10
Scenario 1 Passed.

--- Scenario 2: Price Move, No Cross (10@90 -> 10@95) ---

--- Modifying Order ---
Order ID: 1
Order Update: {'type': 'limit', 'side': 'bid', 'quantity': Decimal('10'), 'price': Decimal('95'), 'trade_id': 'B'}
Scenario 2 Passed.

--- Scenario 3: Qty Increase (10@90 -> 15@90) ---

--- Modifying Order ---
Order ID: 1
Order Update: {'type': 'limit', 'side': 'bid', 'quantity': Decimal('15'), 'price': Decimal('90'), 'trade_id': 'B'}
Scenario 3 Passed.

--- Scenario 4: Qty Decrease (10@90 -> 5@90) ---

--- Modifying Order ---
Order ID: 1
Order Update: {'type': 'limit', 'side': 'bid', 'quantity': Decimal('5'), 'price': Decimal('90'), 'trade_id': 'B'}
Scenario 4

In [9]:
# %run ../gym_continuousDoubleAuction/inspect_latest_episode.py

In [10]:
# %run ../gym_continuousDoubleAuction/analyze_unused.py

In [11]:
from datetime import datetime

now = datetime.now()
formatted = now.strftime("%Y-%m-%d %H:%M:%S")
print(formatted)  # e.g., "2025-12-22 10:30:45"

2025-12-22 03:29:18
